In [45]:
import pandas as pd
import numpy as np
from pathlib import Path
import re
import unicodedata

WS = Path('/workspaces/pokemon')
DATA = WS / 'data'

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

## 1. Load Mapping 34 Tỉnh/Thành Chính Thức

In [46]:
# Load mapping từ file chính thức
mapping_34 = pd.read_csv(DATA / 'vietnam_province_name_mapping.csv')

# Dictionary: old_name -> new_name (merged)
old_to_new = dict(zip(mapping_34['old'], mapping_34['new']))
canonical_34 = sorted(set(mapping_34['new'].unique()))

print(f'✓ Loaded {len(canonical_34)} chính thức tỉnh/thành:')
for i, p in enumerate(canonical_34, 1):
    print(f'  {i:2}. {p}')

✓ Loaded 34 chính thức tỉnh/thành:
   1. An Giang
   2. Bắc Ninh
   3. Cao Bằng
   4. Cà Mau
   5. Gia Lai
   6. Hà Tĩnh
   7. Hưng Yên
   8. Khánh Hoà
   9. Lai Châu
  10. Lào Cai
  11. Lâm Đồng
  12. Lạng Sơn
  13. Nghệ An
  14. Ninh Bình
  15. Phú Thọ
  16. Quảng Ngãi
  17. Quảng Ninh
  18. Quảng Trị
  19. Sơn La
  20. TP. Cần Thơ
  21. TP. Huế
  22. TP. Hà Nội
  23. TP. Hải Phòng
  24. TP. Hồ Chí Minh
  25. TP. Đà Nẵng
  26. Thanh Hóa
  27. Thái Nguyên
  28. Tuyên Quang
  29. Tây Ninh
  30. Vĩnh Long
  31. Điện Biên
  32. Đắk Lắk
  33. Đồng Nai
  34. Đồng Tháp


## 2. Normalization Utilities

In [47]:
def normalize(s: str) -> str:
    """Chuẩn hóa: lowercase, bỏ dấu, collapse spaces."""
    s = str(s).strip().lower()
    # Handle special case: Đ/đ (LATIN CAPITAL/SMALL LETTER D WITH STROKE)
    s = s.replace('đ', 'd').replace('Đ', 'd')
    # Convert various dashes to standard hyphen
    s = s.replace('–', '-').replace('—', '-')
    s = unicodedata.normalize('NFD', s)
    s = ''.join(c for c in s if unicodedata.category(c) != 'Mn')
    s = re.sub(r'\s+', ' ', s)
    s = re.sub(r'[^a-z0-9\s\-]', '', s)
    return s.strip()

# Tạo alias map từ danh sách chính thức 34 tỉnh
alias_map = {}
for canonical in canonical_34:
    norm = normalize(canonical)
    if norm:
        alias_map[norm] = canonical

# Thêm aliases từ cột 'old' (post-merger mapping)
for old, new in old_to_new.items():
    norm = normalize(old)
    if norm and norm not in alias_map:
        alias_map[norm] = new

# Thêm special aliases
alias_map['dac lak'] = 'Đắk Lắk'  # typo for Đắk Lắk
alias_map['dac nong'] = 'Lâm Đồng'  # sáp nhập from Đắk Nông
alias_map['ba ria - vung tau'] = 'TP. Hồ Chí Minh'  # sáp nhập (with hyphen)
alias_map['ba ria vung tau'] = 'TP. Hồ Chí Minh'  # sáp nhập (no hyphen)

print(f'✓ Built alias map: {len(alias_map)} entries')

✓ Built alias map: 66 entries


## 3. Curated District/City Mappings

In [48]:
# Huyện/Thị xã/Thành phố -> Tỉnh chính thức (34)
DISTRICT_MAP = {
    # An Giang
    'tri ton': 'An Giang',
    'tinh bien': 'An Giang',
    'thoai son': 'An Giang',
    'long xuyen': 'An Giang',
    'chau doc': 'An Giang',
    'thanh pho chau doc': 'An Giang',
    'an phu': 'An Giang',
    # Bắc Ninh
    'bac ninh': 'Bắc Ninh',
    'thuan thanh': 'Bắc Ninh',
    'que vo': 'Bắc Ninh',
    'tien du': 'Bắc Ninh',
    'gia binh': 'Bắc Ninh',
    'tu son': 'Bắc Ninh',
    # Cao Bằng
    'cao bang': 'Cao Bằng',
    # Cà Mau
    'ca mau': 'Cà Mau',
    # TP. Cần Thơ
    'can tho': 'TP. Cần Thơ',
    # TP. Đà Nẵng
    'da nang': 'TP. Đà Nẵng',
    # Đắk Lắk
    'dak lak': 'Đắk Lắk',
    'ak lak': 'Đắk Lắk',
    'ac lak': 'Đắk Lắk',
    'dac lak': 'Đắk Lắk',
    # Đắk Nông -> sáp nhập Lâm Đồng
    'dak nong': 'Lâm Đồng',
    'ak nong': 'Lâm Đồng',
    'ac nong': 'Lâm Đồng',
    'dac nong': 'Lâm Đồng',
    # Đồng Tháp
    'cao lanh': 'Đồng Tháp',
    'sa dec': 'Đồng Tháp',
    'thanh pho sa dec': 'Đồng Tháp',
    'lap vo': 'Đồng Tháp',
    'tam nong': 'Đồng Tháp',
    'thap muoi': 'Đồng Tháp',
    'huyen thap muoi': 'Đồng Tháp',
    # Điện Biên
    'dien bien': 'Điện Biên',
    # Gia Lai
    'pleiku': 'Gia Lai',
    'chu pah': 'Gia Lai',
    'chu prong': 'Gia Lai',
    'chu se': 'Gia Lai',
    'ia grai': 'Gia Lai',
    'kbang': 'Gia Lai',
    'kon ka kinh': 'Gia Lai',
    # Hà Tĩnh
    'ha tinh': 'Hà Tĩnh',
    # TP. Hà Nội
    'ha noi': 'TP. Hà Nội',
    # TP. Hải Phòng
    'hai phong': 'TP. Hải Phòng',
    # TP. Hồ Chí Minh
    'ho chi minh': 'TP. Hồ Chí Minh',
    'hcm': 'TP. Hồ Chí Minh',
    'ba ria vung tau': 'TP. Hồ Chí Minh',
    'ba ria  vung tau': 'TP. Hồ Chí Minh',
    'huyen chau thanh': 'TP. Hồ Chí Minh',  # Huyện Châu Thành từ Tiền Giang
    # TP. Huế
    'hue': 'TP. Huế',
    # Hưng Yên
    'hung yen': 'Hưng Yên',
    # Khánh Hoà
    'khanh hoa': 'Khánh Hoà',
    # Lai Châu
    'lai chau': 'Lai Châu',
    'muong khuong': 'Lai Châu',
    'bac ha': 'Lai Châu',
    'bao yen': 'Lai Châu',
    # Lạng Sơn
    'lang son': 'Lạng Sơn',
    # Lào Cai
    'lao cai': 'Lào Cai',
    'sapa': 'Lào Cai',
    'sa pa': 'Lào Cai',
    # Lâm Đồng
    'lam dong': 'Lâm Đồng',
    # Nghệ An
    'nghe an': 'Nghệ An',
    # Ninh Bình
    'ninh binh': 'Ninh Bình',
    # Phú Thọ
    'phu tho': 'Phú Thọ',
    # Quảng Ngãi
    'quang ngai': 'Quảng Ngãi',
    # Quảng Ninh
    'quang ninh': 'Quảng Ninh',
    # Quảng Trị
    'vinh linh': 'Quảng Trị',
    'gio linh': 'Quảng Trị',
    'da krong': 'Quảng Trị',
    'huong hoa': 'Quảng Trị',
    'hai lang': 'Quảng Trị',
    'trieu phong': 'Quảng Trị',
    'quang tri': 'Quảng Trị',
    'con co': 'Quảng Trị',
    # Sơn La
    'son la': 'Sơn La',
    # Tây Ninh
    'go dau': 'Tây Ninh',
    'trang bang': 'Tây Ninh',
    'tan bien': 'Tây Ninh',
    'hoa thanh': 'Tây Ninh',
    # Thái Nguyên
    'thai nguyen': 'Thái Nguyên',
    # Thanh Hóa
    'thanh hoa': 'Thanh Hóa',
    # Tuyên Quang
    'tuyen quang': 'Tuyên Quang',
    # Vĩnh Long
    'vinh long': 'Vĩnh Long',
}

print(f'✓ Loaded {len(DISTRICT_MAP)} curated mappings')

✓ Loaded 82 curated mappings


## 4. Normalization Function

In [49]:
def normalize_province(raw_name: str) -> tuple[str, str]:
    """
    Normalize province name to one of 34 official tỉnh/thành.
    Returns: (canonical_province, status)
    """
    if pd.isna(raw_name) or not str(raw_name).strip():
        return '', 'empty'
    
    raw = str(raw_name).strip()
    norm = normalize(raw)
    
    # 1. Direct alias match
    if norm in alias_map:
        return alias_map[norm], 'direct'
    
    # 2. District match
    if norm in DISTRICT_MAP:
        return DISTRICT_MAP[norm], 'district'
    
    # 3. Try removing common prefixes (place types)
    for prefix in ['vuon quoc gia', 'thanh pho', 'tp', 'thi xa', 'quan', 'huyen', 'thi tran', 'di tich']:
        if norm.startswith(prefix + ' ') or norm.startswith(prefix + '-'):
            rest = norm[len(prefix):].strip().replace('-', ' ')
            if rest in alias_map:
                return alias_map[rest], 'prefix_alias'
            if rest in DISTRICT_MAP:
                return DISTRICT_MAP[rest], 'prefix_district'
    
    return '', 'unmapped'

## 5. Process keyword_mapping.csv

In [50]:
# Load data
km = pd.read_csv(DATA / 'keyword_mapping.csv')
print(f'Input: {len(km)} rows, {km["province"].nunique()} unique provinces')

# Normalize
results = []
for _, row in km.iterrows():
    # If province is NaN/empty, try using normalized_name instead
    province_name = row['province']
    if pd.isna(province_name) or not str(province_name).strip():
        province_name = row.get('normalized_name', '')
    
    canonical, status = normalize_province(province_name)
    results.append({
        'original_province': row['province'],
        'province_normalized': canonical,
        'status': status
    })

norm_df = pd.DataFrame(results)
km_clean = pd.concat([km, norm_df], axis=1)

print(f'\\nOutput: {len(km_clean)} rows')
print(f'Normalized unique provinces: {km_clean[km_clean["province_normalized"] != ""]["province_normalized"].nunique()}')

Input: 967 rows, 98 unique provinces
\nOutput: 967 rows
Normalized unique provinces: 34


## 6. Statistics

In [51]:
# Stats
total = len(km_clean)
mapped = (km_clean['province_normalized'] != '').sum()
unmapped = (km_clean['status'] == 'unmapped').sum()

print('='*60)
print('NORMALIZATION RESULTS')
print('='*60)
print(f'Total rows:           {total:5d}')
print(f'Mapped:               {mapped:5d} ({mapped/total*100:.1f}%)')
print(f'Unmapped:             {unmapped:5d}')
print(f'Original provinces:   {km["province"].nunique():5d}')
print(f'Target provinces:     {34:5d}')
print(f'Achieved provinces:   {km_clean[km_clean["province_normalized"] != ""]["province_normalized"].nunique():5d}')
print('='*60)

# Status breakdown
print('\nStatus breakdown:')
print(km_clean['status'].value_counts())

# Top provinces
print('\nTop 10 provinces by frequency:')
top = km_clean[km_clean['province_normalized'] != '']['province_normalized'].value_counts().head(10)
for p, count in top.items():
    print(f'  {p:30} {count:4d}')

NORMALIZATION RESULTS
Total rows:             967
Mapped:                 967 (100.0%)
Unmapped:                 0
Original provinces:      98
Target provinces:        34
Achieved provinces:      34

Status breakdown:
status
direct             671
prefix_district    266
prefix_alias        18
district            12
Name: count, dtype: int64

Top 10 provinces by frequency:
  Bắc Ninh                         68
  Quảng Ninh                       67
  Lâm Đồng                         53
  TP. Huế                          50
  TP. Đà Nẵng                      49
  TP. Hà Nội                       49
  TP. Hồ Chí Minh                  44
  Phú Thọ                          43
  Khánh Hoà                        42
  Thái Nguyên                      40


## 7. Review Unmapped

In [52]:
unmapped_df = km_clean[km_clean['province_normalized'] == ''].copy()
print(f'\nUnmapped: {len(unmapped_df)}')
if len(unmapped_df) > 0:
    print('\nBy original province:')
    print(unmapped_df['original_province'].value_counts())
else:
    print('✅ All mapped!')


Unmapped: 0
✅ All mapped!


## 8. Export Results

In [53]:
# Export with clean columns only
export_df = km[['row_index', 'original_name', 'normalized_name', 'province']].copy()
export_df['province_normalized'] = km_clean['province_normalized']
export_df['status'] = km_clean['status']

out_clean = DATA / 'keyword_mapping_normalized.csv'
export_df.to_csv(out_clean, index=False)
print(f'✓ Saved to: {out_clean}')

# Unmapped for review
if len(unmapped_df) > 0:
    out_unres = DATA / 'keyword_mapping_unresolved_review.csv'
    unmapped_df[['row_index', 'original_name', 'original_province', 'status']].to_csv(out_unres, index=False)
    print(f'✓ Saved unresolved to: {out_unres}')

✓ Saved to: /workspaces/pokemon/data/keyword_mapping_normalized.csv
